<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Azure-Sentinel-API-Calls" data-toc-modified-id="Azure-Sentinel-API-Calls-1">Azure Sentinel API Calls</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1.0.1">Description</a></span></li><li><span><a href="#Installation-and-imports" data-toc-modified-id="Installation-and-imports-1.0.2">Installation and imports</a></span></li><li><span><a href="#Authentication" data-toc-modified-id="Authentication-1.0.3">Authentication</a></span></li></ul></li></ul></li></ul></div>

# Azure Sentinel API Calls

MSTICpy versions > 0.8.5

### Description

This Notebook provides an example of using the Azure Sentinel API features of MSTICpy in order retrieve specific data from Azure Sentinel

### Installation and imports

In [ ]:
!pip install msticpy --upgrade

In [1]:
from msticpy.data.azure_sentinel import AzureSentinel
import msticpy.nbtools.nbwidgets as widgets
from msticpy.data import data_obfus as mask

### Authentication
The first step to be able to use the features is to call the AzureSentinel class and connect to it. Authentication uses the standardized Azure authentication options of using environment variables, Azure CLI credentials, Managed Identities, and interactive logons.

In [2]:
azs = AzureSentinel()
azs.connect()

Attempting to sign-in with environment variable credentials...


Once connected we need to select an Azure Sentinel workspace to get details from. The easies way to do this is with the get_subscriptions() and get_sentinel_workspaces() functions to select the subscription and workspace you with to connect to. If you already know which workspace you wish to connect to you can skip straight to the other functions and enter these details.

In [24]:
# Query for our subscriptions
subs = azs.get_subscriptions()

# Display subscriptions (masked names) in a pick list
print("Select a subscription:")
sub = widgets.SelectItem(
    item_list=sorted(subs),
    auto_display=True
)

Select a subscription:


In [32]:
# Get the subscription ID
sub_id = subs[subs['Display Name'] == subs_name[sub.value]]['Subscription ID'].iloc[0]

# Query for workspaces in that subscription
workspaces = azs.get_sentinel_workspaces(sub_id)

# Diaplay workspaces in a list
print("Select an Azure Sentinel Workspace:")
ws = widgets.SelectItem(
    item_dict=dict(sorted(workspaces.items())),
    auto_display=True
)

Finding Azure Sentinel Workspaces...
Select an Azure Sentinel Workspace:


Now that we have selected our workspace we can call various functions to get details about content in the workspace. These are typically returned as DataFrames. Below we get a list of hunting queries configured in our workspace.

In [34]:
queries = azs.get_hunting_queries(ws.value)
queries.head().drop(columns=["id", "etag", "name"])

,type,Category,DisplayName,Query,Tags,Version,FunctionAlias,FunctionParameters
0,Microsoft.OperationalInsights/savedSearches,Hunting Queries,Anomalous AAD Account Creation,"\nBehaviorAnalytics\n| where ActionType == ""Ad...","[{'Name': 'description', 'Value': ''}, {'Name'...",2,NaN,NaN
19,Microsoft.OperationalInsights/savedSearches,Hunting Queries,Preview - TI map File entity to VMConnection E...,\nlet dt_lookBack = 1h;\nlet ioc_lookBack = 14...,"[{'Name': 'description', 'Value': ''}, {'Name'...",2,NaN,NaN
28,Microsoft.OperationalInsights/savedSearches,Hunting Queries,Set Hunting Query,Heartbeat | take 1,"[{'Name': 'description', 'Value': 'Test Huntin...",2,NaN,NaN
29,Microsoft.OperationalInsights/savedSearches,Hunting Queries,Uncommon processes/files - bottom 5%,let ProcessCreationEvents=() {\n let pr...,"[{'Name': 'description', 'Value': ''}, {'Name'...",2,NaN,NaN
30,Microsoft.OperationalInsights/savedSearches,Hunting Queries,Test123,SecurityAlert\r\n\t| extend CompromisedHost = ...,"[{'Name': 'description', 'Value': ''}, {'Name'...",2,NaN,NaN


Hunting queries return the raw queries associated with them, this allows us to pass the query directly to a QueryProvider in order to get the results of the hunting query within the notebook.

In [14]:
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
qry_prov = QueryProvider('LogAnalytics')
wkspace = WorkspaceConfig()
qry_prov.connect(wkspace.code_connect_str)
qry_prov.exec_query(queries['Query'].iloc[2])

<IPython.core.display.Javascript object>

,TenantId,SourceSystem,TimeGenerated,MG,ManagementGroupName,SourceComputerId,ComputerIP,Computer,Category,OSType,...,ResourceGroup,ResourceProvider,Resource,ResourceId,ResourceType,ComputerEnvironment,Solutions,VMUUID,Type,_ResourceId
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,OpsManager,2020-04-21 20:08:31.640000+00:00,00000000-0000-0000-0000-000000000001,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,263a788b-6526-4cdc-8ed9-d79402fe4aa0,40.76.43.124,MSTICAlertsWin1,Direct Agent,Windows,...,ASIHUNTOMSWORKSPACERG,Microsoft.Compute,MSTICAlertsWin1,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f...,virtualMachines,Azure,"""security"", ""changeTracking"", ""networkMonitori...",3b8d40a2-33fa-4230-9dc1-794aabdd225c,Heartbeat,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f...


We can also get a list of configured alert rules:

In [11]:
alert_rules = azs.get_alert_rules(ws.value)
alert_rules.head().drop(columns=["id", "etag", "name"])

,type,kind,severity,query,queryFrequency,queryPeriod,triggerOperator,triggerThreshold,suppressionDuration,suppressionEnabled,...,incidentConfiguration.groupingConfiguration.enabled,incidentConfiguration.groupingConfiguration.reopenClosedIncident,incidentConfiguration.groupingConfiguration.lookbackDuration,incidentConfiguration.groupingConfiguration.entitiesMatchingMethod,incidentConfiguration.groupingConfiguration.groupByEntities,eventGroupingSettings.aggregationKind,productFilter,severitiesFilter,displayNamesFilter,displayNamesExcludeFilter
0,Microsoft.SecurityInsights/alertRules,Scheduled,Medium,OfficeActivity| take 1,PT5H,PT5H,GreaterThan,5.0,PT5H,False,...,False,False,PT5H,All,[],SingleAlert,NaN,NaN,NaN,NaN
1,Microsoft.SecurityInsights/alertRules,Scheduled,Informational,"let EventNameList = dynamic([""AttachUserPolic...",PT5H,P10D,GreaterThan,0.0,PT5H,False,...,False,False,PT5M,All,[],NaN,NaN,NaN,NaN,NaN
2,Microsoft.SecurityInsights/alertRules,MLBehaviorAnalytics,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Microsoft.SecurityInsights/alertRules,Scheduled,Medium,let starttime = 14d;\r\nlet endtime = 1d;\r\nl...,PT5H,P7D,GreaterThan,0.0,PT5H,False,...,False,False,PT5M,All,[],NaN,NaN,NaN,NaN,NaN
4,Microsoft.SecurityInsights/alertRules,Scheduled,Low,let starttime = 14d;\r\nlet midtime = 8d;\r\nl...,PT5H,P7D,GreaterThan,0.0,PT5H,False,...,False,False,PT5M,All,[],NaN,NaN,NaN,NaN,NaN


We can also get a list of saved bookmarks. To see the events these bookmarks relate to you can pass the query value to a QueryProvider.

In [12]:
bkmarks = azs.get_bookmarks(ws.value)
bkmarks.head().drop(columns=["id", "etag", "name"])

,type,displayName,created,updated,eventTime,labels,query,queryResult,queryStartTime,queryEndTime,...,createdBy.email,createdBy.name,updatedBy.objectId,updatedBy.email,updatedBy.name,incidentInfo.incidentId,incidentInfo.title,incidentInfo.relationName,incidentInfo.severity,notes
0,Microsoft.SecurityInsights/Bookmarks,SecurityAlert - ef11936a7a66 (3),2020-04-27T16:55:02.8309802+00:00,2020-04-27T16:55:02.8309802+00:00,2020-04-27T16:55:02.8309802+00:00,[],SecurityAlert\n| take 10\n,"{""TenantId"":""52b1ab41-869e-4138-9e40-2a4457f09...",2020-04-26T16:54:49.439+00:00,2020-04-27T16:54:49.439+00:00,...,pagoudjo@microsoft.com,100320004CA0C08B,ddf6375d-b314-4369-b56d-8d787b321b0e,pagoudjo@microsoft.com,100320004CA0C08B,9f63476e-3605-4377-a415-7626fdda46f3,SecurityAlert - ef11936a7a66 (3),4c0a9344-c06d-49d9-b831-bc18bf2cc77a,Medium,NaN
1,Microsoft.SecurityInsights/Bookmarks,SecurityAlert - eb067cbefcd3 (2),2020-04-26T04:40:45.4130364+00:00,2020-04-26T04:40:45.4130364+00:00,2020-04-26T04:40:45.4130364+00:00,[],SecurityAlert\n| take 10\n,"{""TenantId"":""52b1ab41-869e-4138-9e40-2a4457f09...",2020-04-25T04:40:38.626+00:00,2020-04-26T04:40:38.626+00:00,...,pagoudjo@microsoft.com,100320004CA0C08B,ddf6375d-b314-4369-b56d-8d787b321b0e,pagoudjo@microsoft.com,100320004CA0C08B,None,None,None,None,NaN
2,Microsoft.SecurityInsights/Bookmarks,SecurityEvent - Test (1),2020-04-17T17:58:55.0675533+00:00,2020-04-17T17:58:55.0675533+00:00,2020-04-17T17:58:55.0675533+00:00,[testing],SecurityEvent\n| take 10\n,"{""TimeGenerated"":""2020-04-17T07:55:51.003Z"",""A...",2020-04-16T17:57:54.712+00:00,2020-04-17T17:57:54.712+00:00,...,pagoudjo@microsoft.com,100320004CA0C08B,ddf6375d-b314-4369-b56d-8d787b321b0e,pagoudjo@microsoft.com,100320004CA0C08B,383ff7bb-23b8-4616-af93-bcd04efe89cf,SecurityEvent - Investigate issue,abc6d40d-0b0e-4ada-b829-bafbb540eb39,Medium,dfdfd
3,Microsoft.SecurityInsights/Bookmarks,SecurityAlert - eb067cbefcd3 (1),2020-04-26T04:40:43.6423939+00:00,2020-04-26T04:40:43.6423939+00:00,2020-04-26T04:40:43.6423939+00:00,[],SecurityAlert\n| take 10\n,"{""TenantId"":""52b1ab41-869e-4138-9e40-2a4457f09...",2020-04-25T04:40:38.626+00:00,2020-04-26T04:40:38.626+00:00,...,pagoudjo@microsoft.com,100320004CA0C08B,ddf6375d-b314-4369-b56d-8d787b321b0e,pagoudjo@microsoft.com,100320004CA0C08B,None,None,None,None,NaN
4,Microsoft.SecurityInsights/Bookmarks,SecurityAlert - eb067cbefcd3,2020-04-26T04:40:42.2198094+00:00,2020-04-26T04:40:42.2198094+00:00,2020-04-26T04:40:42.2198094+00:00,[],SecurityAlert\n| take 10\n,"{""TenantId"":""52b1ab41-869e-4138-9e40-2a4457f09...",2020-04-25T04:40:38.626+00:00,2020-04-26T04:40:38.626+00:00,...,pagoudjo@microsoft.com,100320004CA0C08B,ddf6375d-b314-4369-b56d-8d787b321b0e,pagoudjo@microsoft.com,100320004CA0C08B,None,None,None,None,NaN
